# 6. Perceptron

In [1]:
from random import seed, randrange

from functions import *

### Define functions
* Perceptron, or single neuron, using SGD

In [2]:
# Make a prediction with weights based on perceptron (single neuron)
def predict_perceptron(row, weights):
    activation = weights[0]
    # Loop over all input columns except last (i.e. actual output)
    for i in range(len(row)-1):
        # For each column, multiply input by coefficient and update activation
        activation += weights[i + 1] * row[i]
    # Neuron fires if activation function >= 0, does not fire if < 0
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using Stochastic Gradient Descent (SGD)
# Inputs: learning rate, number of epochs, (mini) batch size with default 1
# Outputs: Perceptron weights
def train_weights_perceptron(train, l_rate, n_epoch, b = 1):
    # Initialize all weights to 0.0, including intercept (bias) at w0
    weights = [0.0 for col in range(len(train[0]))]
    for epoch in range(n_epoch):
        # Overall epoch squared error
        sum_error = 0.0
        # Counters to track batch index and overall train set index
        i, j = 0, 0
        # Average SUM(h - y) * xi over weights i, to multiply batch by learning rate
        adjustments = [0.0 for col in range(len(train[0]))]

        for row in train:
            # Increment batch and train set indices
            i += 1
            j += 1
            # Calculate prediction and error against actual based on latest weights
            prediction = predict_perceptron(row, weights)
            error = row[-1] - prediction
            # Square and add to overall epoch error
            sum_error += error**2

            # Add to batch weight adjustments based on corresponding inputs (except bias)
            adjustments[0] += error
            for k in range(len(row)-1):
                # Begin with first weight, skipping intercept at k = 0
                # First column in row corresponds to second weight, since first is intercept
                adjustments[k + 1] += error * row[k]

            # Adjust all weights if row is last in batch or overall train set
            if i == b or j == len(train):
                #print('Adjustments made at row # ', str(j))
                for k in range(len(adjustments)):
                    weights[k] = weights[k] + l_rate * (1/i) * adjustments[k]
                    # i = b when at end of batch, but i < b if train set ends mid-match
                # Reset batch adjustments and batch index counter
                adjustments = [0.0 for col in range(len(train[0]))]
                i = 0

        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    # Return weighs after iterating through all epochs -- and rows within each
    return weights

# Perceptron Algorithm with Stochastic Gradient Descent
# Input: train and test sets, learning rate, number of epochs, and batch size
# Output: list of predictions to compare against test data actual outputs
def perceptron(train, test, l_rate, n_epoch, b_size = 1):
    predictions = list()
    # Get Perceptron weights using SGD and input parameters
    weights = train_weights_perceptron(train, l_rate, n_epoch, b_size)
    # Predict output for all test rows, using Perceptron model built on train set
    for row in test:
        # Predict output based on test set inputs and weights estimated using train set
        prediction = predict_perceptron(row, weights)
        predictions.append(prediction)
    # Return predicted outputs in test set using train set weight estimates
    return predictions

### Testing perceptron on contrived dataset

In [3]:
print("\nTesting prediction method using contrived dataset:")
# Contrived dataset: x1, x2, y
dataset =   [[2.7810836,2.550537003,0],
            [1.465489372,2.362125076,0],
            [3.396561688,4.400293529,0],
            [1.38807019,1.850220317,0],
            [3.06407232,3.005305973,0],
            [7.627531214,2.759262235,1],
            [5.332441248,2.088626775,1],
            [6.922596716,1.77106367,1],
            [8.675418651,-0.242068655,1],
            [7.673756466,3.508563011,1]]
# Hardcoded test weights: intercept (bias), x1, x2
weights = [-0.1, 0.20653640140000007, -0.23418117710000003]
# I.e. activation = -0.1 + 0.206*x1 - 0.234*x2
# Iterate over dataset rows, making and printing prediction
for row in dataset:
    prediction = predict_perceptron(row, weights)
    print("Expected=%d, Predicted=%d" % (row[-1], prediction))

print("\nTesting Stochastic Gradient Descent using contrived dataset:")
l_rate = 0.1
n_epoch = 5
weights = train_weights_perceptron(dataset, l_rate, n_epoch)
print(weights)


Testing prediction method using contrived dataset:
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1

Testing Stochastic Gradient Descent using contrived dataset:
>epoch=0, lrate=0.100, error=2.000
>epoch=1, lrate=0.100, error=1.000
>epoch=2, lrate=0.100, error=0.000
>epoch=3, lrate=0.100, error=0.000
>epoch=4, lrate=0.100, error=0.000
[-0.1, 0.20653640140000007, -0.23418117710000003]


### Testing perceptron on Sonar dataset

In [4]:
print("\n Training Perceptron model using SGD on Sonar dataset:")
# Using k-fold cross validation and dynamic algorithm evaluation method
seed(1)
# Load dataset and convert all columns from string to float (numeric type)
dataset = load_csv('data/sonar.all-data.csv')
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# Convert last column from float to int, since binary classification output
str_column_to_int(dataset, len(dataset[0])-1)
# Splits dataset into 3 folds; 3 iterations of training on 2 folds and testing on 1
# 208 rows across 3 folds = 69.3 ~ 69 records per fold
n_folds = 3
l_rate = 0.1
# For each fold, the weights are exposed to the full train set and adjusted 500 times
n_epoch = 500
# In each epoch, weights are adjusted (number of rows / batch size) times
# Batch Gradient Descent if b = 138 = train size (2 folds, i.e. batch = all rows)
# Mini-batch if b < 138
b_size = 10

# l_rate and n_epoch are passed to Perceptron algorithm as *args by evaluate_algorithm
scores = evaluate_algorithm(dataset, perceptron, 3, accuracy_metric, l_rate, n_epoch, b_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))



 Training Perceptron model using SGD on Sonar dataset:
Loaded data file data/sonar.all-data.csv with 208 rows and 61 columns.
>epoch=0, lrate=0.100, error=68.000
>epoch=1, lrate=0.100, error=65.000
>epoch=2, lrate=0.100, error=46.000
>epoch=3, lrate=0.100, error=56.000
>epoch=4, lrate=0.100, error=59.000
>epoch=5, lrate=0.100, error=41.000
>epoch=6, lrate=0.100, error=52.000
>epoch=7, lrate=0.100, error=46.000
>epoch=8, lrate=0.100, error=42.000
>epoch=9, lrate=0.100, error=40.000
>epoch=10, lrate=0.100, error=38.000
>epoch=11, lrate=0.100, error=36.000
>epoch=12, lrate=0.100, error=41.000
>epoch=13, lrate=0.100, error=44.000
>epoch=14, lrate=0.100, error=47.000
>epoch=15, lrate=0.100, error=44.000
>epoch=16, lrate=0.100, error=38.000
>epoch=17, lrate=0.100, error=42.000
>epoch=18, lrate=0.100, error=41.000
>epoch=19, lrate=0.100, error=35.000
>epoch=20, lrate=0.100, error=35.000
>epoch=21, lrate=0.100, error=34.000
>epoch=22, lrate=0.100, error=27.000
>epoch=23, lrate=0.100, error=37

>epoch=300, lrate=0.100, error=16.000
>epoch=301, lrate=0.100, error=24.000
>epoch=302, lrate=0.100, error=22.000
>epoch=303, lrate=0.100, error=24.000
>epoch=304, lrate=0.100, error=24.000
>epoch=305, lrate=0.100, error=24.000
>epoch=306, lrate=0.100, error=23.000
>epoch=307, lrate=0.100, error=19.000
>epoch=308, lrate=0.100, error=23.000
>epoch=309, lrate=0.100, error=26.000
>epoch=310, lrate=0.100, error=23.000
>epoch=311, lrate=0.100, error=25.000
>epoch=312, lrate=0.100, error=20.000
>epoch=313, lrate=0.100, error=24.000
>epoch=314, lrate=0.100, error=28.000
>epoch=315, lrate=0.100, error=19.000
>epoch=316, lrate=0.100, error=24.000
>epoch=317, lrate=0.100, error=18.000
>epoch=318, lrate=0.100, error=17.000
>epoch=319, lrate=0.100, error=26.000
>epoch=320, lrate=0.100, error=25.000
>epoch=321, lrate=0.100, error=26.000
>epoch=322, lrate=0.100, error=24.000
>epoch=323, lrate=0.100, error=21.000
>epoch=324, lrate=0.100, error=29.000
>epoch=325, lrate=0.100, error=23.000
>epoch=326, 

>epoch=56, lrate=0.100, error=30.000
>epoch=57, lrate=0.100, error=28.000
>epoch=58, lrate=0.100, error=22.000
>epoch=59, lrate=0.100, error=21.000
>epoch=60, lrate=0.100, error=29.000
>epoch=61, lrate=0.100, error=26.000
>epoch=62, lrate=0.100, error=36.000
>epoch=63, lrate=0.100, error=18.000
>epoch=64, lrate=0.100, error=17.000
>epoch=65, lrate=0.100, error=43.000
>epoch=66, lrate=0.100, error=18.000
>epoch=67, lrate=0.100, error=20.000
>epoch=68, lrate=0.100, error=20.000
>epoch=69, lrate=0.100, error=29.000
>epoch=70, lrate=0.100, error=30.000
>epoch=71, lrate=0.100, error=30.000
>epoch=72, lrate=0.100, error=30.000
>epoch=73, lrate=0.100, error=22.000
>epoch=74, lrate=0.100, error=26.000
>epoch=75, lrate=0.100, error=20.000
>epoch=76, lrate=0.100, error=30.000
>epoch=77, lrate=0.100, error=25.000
>epoch=78, lrate=0.100, error=26.000
>epoch=79, lrate=0.100, error=27.000
>epoch=80, lrate=0.100, error=18.000
>epoch=81, lrate=0.100, error=25.000
>epoch=82, lrate=0.100, error=27.000
>

>epoch=319, lrate=0.100, error=20.000
>epoch=320, lrate=0.100, error=22.000
>epoch=321, lrate=0.100, error=28.000
>epoch=322, lrate=0.100, error=18.000
>epoch=323, lrate=0.100, error=15.000
>epoch=324, lrate=0.100, error=24.000
>epoch=325, lrate=0.100, error=17.000
>epoch=326, lrate=0.100, error=26.000
>epoch=327, lrate=0.100, error=15.000
>epoch=328, lrate=0.100, error=22.000
>epoch=329, lrate=0.100, error=23.000
>epoch=330, lrate=0.100, error=23.000
>epoch=331, lrate=0.100, error=20.000
>epoch=332, lrate=0.100, error=11.000
>epoch=333, lrate=0.100, error=24.000
>epoch=334, lrate=0.100, error=22.000
>epoch=335, lrate=0.100, error=14.000
>epoch=336, lrate=0.100, error=19.000
>epoch=337, lrate=0.100, error=19.000
>epoch=338, lrate=0.100, error=16.000
>epoch=339, lrate=0.100, error=20.000
>epoch=340, lrate=0.100, error=27.000
>epoch=341, lrate=0.100, error=18.000
>epoch=342, lrate=0.100, error=22.000
>epoch=343, lrate=0.100, error=20.000
>epoch=344, lrate=0.100, error=15.000
>epoch=345, 

>epoch=110, lrate=0.100, error=22.000
>epoch=111, lrate=0.100, error=27.000
>epoch=112, lrate=0.100, error=27.000
>epoch=113, lrate=0.100, error=30.000
>epoch=114, lrate=0.100, error=28.000
>epoch=115, lrate=0.100, error=23.000
>epoch=116, lrate=0.100, error=27.000
>epoch=117, lrate=0.100, error=21.000
>epoch=118, lrate=0.100, error=32.000
>epoch=119, lrate=0.100, error=26.000
>epoch=120, lrate=0.100, error=16.000
>epoch=121, lrate=0.100, error=21.000
>epoch=122, lrate=0.100, error=27.000
>epoch=123, lrate=0.100, error=21.000
>epoch=124, lrate=0.100, error=22.000
>epoch=125, lrate=0.100, error=21.000
>epoch=126, lrate=0.100, error=19.000
>epoch=127, lrate=0.100, error=36.000
>epoch=128, lrate=0.100, error=17.000
>epoch=129, lrate=0.100, error=22.000
>epoch=130, lrate=0.100, error=18.000
>epoch=131, lrate=0.100, error=28.000
>epoch=132, lrate=0.100, error=17.000
>epoch=133, lrate=0.100, error=29.000
>epoch=134, lrate=0.100, error=23.000
>epoch=135, lrate=0.100, error=17.000
>epoch=136, 

>epoch=329, lrate=0.100, error=16.000
>epoch=330, lrate=0.100, error=11.000
>epoch=331, lrate=0.100, error=17.000
>epoch=332, lrate=0.100, error=19.000
>epoch=333, lrate=0.100, error=23.000
>epoch=334, lrate=0.100, error=16.000
>epoch=335, lrate=0.100, error=22.000
>epoch=336, lrate=0.100, error=19.000
>epoch=337, lrate=0.100, error=15.000
>epoch=338, lrate=0.100, error=19.000
>epoch=339, lrate=0.100, error=16.000
>epoch=340, lrate=0.100, error=18.000
>epoch=341, lrate=0.100, error=25.000
>epoch=342, lrate=0.100, error=15.000
>epoch=343, lrate=0.100, error=15.000
>epoch=344, lrate=0.100, error=13.000
>epoch=345, lrate=0.100, error=19.000
>epoch=346, lrate=0.100, error=30.000
>epoch=347, lrate=0.100, error=13.000
>epoch=348, lrate=0.100, error=16.000
>epoch=349, lrate=0.100, error=14.000
>epoch=350, lrate=0.100, error=25.000
>epoch=351, lrate=0.100, error=16.000
>epoch=352, lrate=0.100, error=14.000
>epoch=353, lrate=0.100, error=17.000
>epoch=354, lrate=0.100, error=13.000
>epoch=355, 